<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
# подготавливаем рабочее пространство,
# загружаем библиотеки
!pip install lightgbm
!pip install spacy
!spacy download en
import spacy

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import re

from sklearn.pipeline import make_pipeline

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords as nltk_stopwords
nltk.download('stopwords')
nltk.download('wordnet')

stopwords = set(nltk_stopwords.words('english'))

from lightgbm import LGBMClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import (
    train_test_split,
    cross_val_score,
    GridSearchCV
)
from sklearn.metrics import (
    f1_score, make_scorer,
    precision_score,
    recall_score,
    accuracy_score,
    confusion_matrix,
    roc_auc_score,
    roc_curve
)
from sklearn.feature_extraction.text import TfidfVectorizer

from tqdm import tqdm
tqdm.pandas()

# настройки
# import warnings
# warnings.filterwarnings("ignore")

# константы
RANDOM_STATE = 42

2023-03-13 19:27:44.300292: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2023-03-13 19:27:44.300316: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-13 19:27:46.479968: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2023-03-13 19:27:46.480559: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cublas64_11.dll'; dlerror: cublas64_11.dll not found
2023-03-13 19:27:46.481132: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cublasLt64_11.dll'; dlerror: cublasLt64_11.dll not found
2023-03-13 19:27:46.481700: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cu

  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.5.0/en_core_web_sm-3.5.0-py3-none-any.whl (12.8 MB)
[!] As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use
the full pipeline package name 'en_core_web_sm' instead.
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\matro\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\matro\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# подгрузим наш датасет
try:
    df = pd.read_csv('toxic_comments.csv')
except:
    df = pd.read_csv('/datasets/toxic_comments.csv')

In [3]:
# удалим лишний столбец и вывидем первые 5 строк
df = df.drop('Unnamed: 0', axis=1)
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [4]:
# посмотрим на размер датасета
df.shape

(159292, 2)

In [5]:
# скопируем наш датасет в переменную corpus для дальнейшей работы
corpus = df.copy()
corpus['toxic'].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

In [6]:
# лемматизируем наши тексты оставив лишь необходимое
nlp = spacy.load("en_core_web_sm")

def lemmatize_text(text):
    text = text.lower()
    doc = nlp(text)
    lemm_text = " ".join([token.lemma_ for token in doc])
    cleared_text = re.sub(r'[^a-zA-Z]', ' ', lemm_text) 
    return " ".join(cleared_text.split())

corpus['lemm_text'] = corpus['text'].progress_apply(lemmatize_text)

corpus = corpus.drop('text', axis=1)

100%|██████████| 159292/159292 [27:49<00:00, 95.40it/s]  


In [7]:
# посмотрим на результат
corpus.head()

,toxic,lemm_text
0,0,explanation why the edit make under my usernam...
1,0,d aww he match this background colour I be see...
2,0,hey man I be really not try to edit war it be ...
3,0,more I can not make any real suggestion on imp...
4,0,you sir be my hero any chance you remember wha...


In [8]:
# делим данные на признаки и таргет
X = corpus['lemm_text']
y = corpus['toxic']

# делим выборки на обучающие и тестовые
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=RANDOM_STATE
)

In [9]:
# проверим размеры наших выборок
display(f'Размер тренировочной выборки: {X_train.shape[0]} строк')
display(f'Размер тренировочного целевого признака: {y_train.shape[0]} строк')
display(f'Размер тестовой выборки: {X_test.shape[0]} строки')
display(f'Размер тестового целевого признака: {y_test.shape[0]} строки')

'Размер тренировочной выборки: 143362 строк'

'Размер тренировочного целевого признака: 143362 строк'

'Размер тестовой выборки: 15930 строки'

'Размер тестового целевого признака: 15930 строки'

## Обучение

In [11]:
# с помощью make_pipeline напишем функцию для векторизации и подбора параметров через grid
def pipe(model, params):
    pipeline = make_pipeline(TfidfVectorizer(stop_words=list(stopwords)), model)
    grid = GridSearchCV(pipeline,
                        param_grid=params,
                        n_jobs=-1,
                        verbose=1,
                        scoring='f1',
                        cv=2)
    return grid


# напишем функцию для обучения модели с подобранными параметрами и выводом лучшего f1_score
def fit_best_model(model, X, y):
    model.fit(X, y)
    best_score = model.best_score_
    best_model = model.best_estimator_
    display(f'Параметры лучшей модели: {model.best_params_}')
    display(f'F1 лучшей модели: {best_score}')
    return best_model, best_score

In [12]:
%%time
# подберем параметры и выведем на экран f1 для модели LogisticRegression

params_LR = {'logisticregression__C': np.arange(5, 15, 1)}

grid_LR = pipe(LogisticRegression(random_state=RANDOM_STATE, max_iter=1000, class_weight='balanced'), params_LR)

model_LR, best_score_LR = fit_best_model(grid_LR, X_train, y_train)

Fitting 2 folds for each of 10 candidates, totalling 20 fits


"Параметры лучшей модели: {'logisticregression__C': 10}"

'F1 лучшей модели: 0.7639288339441943'

CPU times: total: 21.4 s
Wall time: 1min 30s


In [13]:
%%time
# подберем параметры и выведем на экран f1 для модели DecisionTreeClassifier

params_DTC = {'decisiontreeclassifier__max_depth': np.arange(2, 51, 2)}

grid_DTC = pipe(DecisionTreeClassifier(random_state=RANDOM_STATE), params_DTC)

model_DTC, best_score_DTC = fit_best_model(grid_DTC, X_train, y_train)

Fitting 2 folds for each of 25 candidates, totalling 50 fits


"Параметры лучшей модели: {'decisiontreeclassifier__max_depth': 50}"

'F1 лучшей модели: 0.7053280558652326'

CPU times: total: 36.8 s
Wall time: 2min 38s


In [14]:
%%time
# подберем параметры и выведем на экран f1 для модели LGBMClassifier

params_LGBMC = {'lgbmclassifier__num_leaves': [100, 150, 200], 
                'lgbmclassifier__learning_rate': [.1, .2]}

grid_LGBMC = pipe(LGBMClassifier(random_state=RANDOM_STATE), params_LGBMC)

model_LGBMC, best_score_LGBMC = fit_best_model(grid_LGBMC, X_train, y_train)

Fitting 2 folds for each of 6 candidates, totalling 12 fits


"Параметры лучшей модели: {'lgbmclassifier__learning_rate': 0.1, 'lgbmclassifier__num_leaves': 100}"

'F1 лучшей модели: 0.7723774385185529'

CPU times: total: 3min 5s
Wall time: 4min 53s


In [15]:
# выведем на экран таблицу с итогами исследования
models = ['LogisticRegression', 'DecisionTreeClassifier', 'LGBMClassifier']
table = pd.DataFrame(np.array([best_score_LR, best_score_DTC, best_score_LGBMC]), index=models)
table.set_axis(['f1'], axis=1, inplace=True)

table

,f1
LogisticRegression,0.763929
DecisionTreeClassifier,0.705328
LGBMClassifier,0.772377


In [16]:
# проверим f1 на тестовой выборке и моделе model_LGBMC
predict_LGBMC = model_LGBMC.predict(X_test)
print(f'f1 на тестовой выборке для модели LGBMClassifier: {f1_score(y_test, predict_LGBMC)}')

f1 на тестовой выборке для модели LGBMClassifier: 0.782608695652174


## BERT

Подготовим тексты с помощью предобученной на токсичных комментариях BERT. И посчитаем метрику f1 на предсказаниях линейной модели.

In [18]:
# загружаем библиотеки
import torch
import transformers
from tqdm import notebook

In [19]:
# загружаем tokenizer и model для предобученной на токсик текстах нейросети bert
tokenizer = transformers.BertTokenizer.from_pretrained('unitary/toxic-bert')
model = transformers.BertModel.from_pretrained('unitary/toxic-bert')

Some weights of the model checkpoint at unitary/toxic-bert were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [20]:
# оставим 1000 записей из датасета чтобы код быстрее отработал
df_for_bert = df.sample(1000).reset_index(drop=True)

In [21]:
# переведем тексты в токены
tokenized = df_for_bert['text'].apply((lambda x: tokenizer.encode(x, 
                                                                  add_special_tokens=True, 
                                                                  max_length=512, 
                                                                  truncation=True)))

In [22]:
# в меньших строках заполним пропуски нулями чтобы все имели одну длину
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(1000, 512)

In [23]:
# проверим доступность видеочипа
torch.cuda.is_available()

False

In [24]:
# обработаем датасет батчами по 100
batch_size = 100
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].numpy())

  0%|          | 0/10 [00:00<?, ?it/s]

In [25]:
# определим признаки и таргет
X_bert = np.concatenate(embeddings)
y_bert = df_for_bert['toxic']

In [26]:
# делим на train и test
X_train_bert, X_test_bert, y_train_bert, y_test_bert = train_test_split(X_bert, 
                                                                        y_bert, 
                                                                        test_size=0.2, 
                                                                        random_state=RANDOM_STATE)

In [27]:
%%time
# подберем лучшие параметры для модели LogisticRegression
# выведем на экран лучшие параметры и f1

LR = LogisticRegression(random_state=RANDOM_STATE, max_iter=1000, class_weight='balanced')

params_LR = {
    'C': np.arange(5, 15, 1)
}

grid_LR_bert = GridSearchCV(LR,
                            param_grid=params_LR,
                            cv=2,
                            scoring='f1',
                            verbose=1,
                            n_jobs=-1)

grid_LR_bert.fit(X_train_bert, y_train_bert)

model_LR_bert = grid_LR_bert.best_estimator_
LR_bet_best_score = grid_LR_bert.best_score_

display(grid_LR_bert.best_params_)
display(LR_bet_best_score)

Fitting 2 folds for each of 10 candidates, totalling 20 fits


{'C': 5}

0.8803571428571428

CPU times: total: 469 ms
Wall time: 3.09 s


In [28]:
# проверим f1 на тестовой выборке и моделе model_LR_bert
predict_LR_bert = model_LR_bert.predict(X_test_bert)
print(f'f1 на тестовой выборке для модели LGBMClassifier: {f1_score(y_test_bert, predict_LR_bert)}')

f1 на тестовой выборке для модели LGBMClassifier: 0.9500000000000001


## Выводы

Модель LGBMClassifier решает поставленную задачу, с ее помощью мы преодалели границу f1=0.75. Из-за дисбаланса классов мы очень хорошо предсказуем больший и хуже меньший класс, смещение приоритета меньшему классу не принесло больших отличий от текущего результата.

Отдельно подготовили признаки с помощью BERT, взяли из датасета 1000 строк случайным образом. Результат линейной модели на признаках подготовленных BERT на тесте f1 = 0.95. Bert значительно превзошла стандартные tfidf методы.